In [1]:
import numpy as np
import pandas as pd

from score_teams import score_drives
from score_teams import aggregate_game_drives, aggregate_season_drives

In [2]:
start_season = 2009
end_season = 2018

In [9]:
ddf = score_drives(start_season, end_season)
ddf['month'] = ddf['game_id'].map(lambda x: int(str(x)[4:6]))
ddf['season'] = ddf['game_id'].map(
    lambda x: int(str(x)[:4]) if int(str(x)[4:6]) > 8 else int(str(x)[:4]) -1
)

ddf['day'] = ddf['game_id'].map(lambda x: int(str(x)[6:8]))

ddf = ddf.loc[(ddf['month'] > 2) | ((ddf['month'] == 1) & (ddf['day'] < 4))].copy()

ddf.head()

,away_team,defensive_team,drive_time,end_quarter,end_time,first_play_desc,game_id,home_score_diff_last_quarter,home_team,last_play_desc,...,season,start_opp_expected_yard_line_bin,end_opp_expected_yard_line_bin,expected_points_opp_from_start,expected_points_opp_from_end,field_position_points,drive_score,nfl_avg_score,month,day
0,TEN,TEN,1:44,1,13:16,R.Bironas kicks 67 yards from TEN 30 to PIT 3....,2009091000,0,PIT,(13:27) (Punt formation) D.Sepulveda punts 54 ...,...,2009,20-30,20-30,1.627169,1.627169,0.000000,-2.244270,2.244270,9,10
1,TEN,PIT,1:52,1,11:24,(13:16) C.Johnson up the middle to TEN 2 for n...,2009091000,0,PIT,(11:34) (Punt formation) C.Hentrich punts 50 y...,...,2009,30-40,50-60,1.899039,2.453221,-0.554182,-1.508551,0.954370,9,10
2,TEN,TEN,3:04,1,08:20,(11:24) B.Roethlisberger pass short right to M...,2009091000,0,PIT,(8:25) (Punt formation) D.Sepulveda punts 30 y...,...,2009,20-30,20-30,1.627169,1.627169,0.000000,-2.452806,2.452806,9,10
3,TEN,PIT,1:36,1,06:44,(8:20) C.Johnson right end to TEN 43 for 32 ya...,2009091000,0,PIT,(6:48) (Field Goal formation) R.Bironas 37 yar...,...,2009,30-40,20-30,1.899039,1.627169,0.271870,1.594445,1.429887,9,10
4,TEN,TEN,1:55,1,04:49,(6:44) R.Mendenhall up the middle to PIT 28 fo...,2009091000,0,PIT,(5:02) (Punt formation) D.Sepulveda punts 53 y...,...,2009,20-30,30-40,1.627169,1.899039,-0.271870,-1.814142,1.542272,9,10


In [10]:
point_totals = ddf.groupby(['offensive_team', 'season'], as_index=False).agg({'points': 'sum', 'game_id': 'nunique'})
point_totals['points_per_game'] = point_totals['points'] / point_totals['game_id']
point_totals = point_totals.sort_values('points_per_game', ascending=False)

point_totals = point_totals.reset_index(drop=True)
point_totals.head(20)

,offensive_team,season,points,game_id,points_per_game
0,DEN,2013,550.910437,16,34.431902
1,GB,2011,515.678404,16,32.229900
2,KC,2018,515.424207,16,32.214013
3,NE,2012,513.517172,16,32.094823
4,NO,2011,512.764710,16,32.047794
5,ATL,2016,483.293075,16,30.205817
6,LA,2018,482.338497,16,30.146156
7,NO,2018,481.982567,16,30.123910
8,NE,2011,474.944121,16,29.684008
9,NE,2010,473.079054,16,29.567441


In [14]:
point_totals[point_totals['game_id'] == 17]

,offensive_team,season,points,game_id,points_per_game
47,BAL,2014,418.054142,17,24.591420
69,PIT,2014,398.280766,17,23.428280
169,CAR,2014,336.848480,17,19.814616
250,ARI,2014,279.416933,17,16.436290


In [5]:
ddf.loc[(ddf['offensive_team'] == 'KC') & (ddf['season'] == 2018), 'game_id'].unique().shape

(18,)

In [6]:
gdf = aggregate_game_drives(2009, 2018, 'offensive_team', opponent_strength=True)

gdf.head()

,game_id,offensive_team,defensive_team,season,drive_score,opponent_adjustment,adj_drive_score
2425,2013111011,NO,DAL,2013,3.749826,0.552189,3.197637
1271,2011102311,NO,IND,2011,3.593545,0.281988,3.311556
2768,2014092800,BAL,CAR,2014,3.464702,0.140448,3.324254
2771,2014092801,GB,CHI,2014,3.353697,0.503703,2.849994
3283,2015092706,NE,JAC,2015,3.174108,0.341769,2.832339


In [7]:
sodf = aggregate_season_drives(2009, 2018, 'offensive_team', opponent_strength=True)

sodf.head()

,season,offensive_team,drive_score,adj_drive_score
83,2011,NO,1.381990,1.237862
306,2018,KC,1.295298,1.097456
114,2012,NE,1.022149,1.049502
50,2010,NE,0.849787,1.043602
227,2016,ATL,1.162691,1.031547


In [8]:
sddf = aggregate_season_drives(2009, 2018, 'defensive_team', opponent_strength=True)

sddf.head()

,season,defensive_team,drive_score,adj_drive_score
213,2015,NO,0.982522,0.864662
44,2010,HOU,0.652083,0.588802
68,2011,CAR,0.676545,0.568141
272,2017,IND,0.411305,0.548394
247,2016,NO,0.643151,0.530140
